In [ ]:
import torch
import torchvision
import cv2
import csv
import torchvision.transforms as transforms
import tensorflow as tf
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import heapq
from keras.models import load_model
import glob
import pandas as pd
from tqdm import tqdm

import sys
    "sys.path.insert(0,'/data/src/')\n",
from data import LungDataset, blend, Pad, Crop, Resize
from models import UNet, PretrainedUNet
from metrics import jaccard, dice, auc, precision, recall, accuracy

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# The outputs of the models will be saved in resultFolder. 
resultFolder="/Users/esmasert/Desktop/PaperRelated/HeatMap/HeatmapInference/results/CV1/"
testImageFolder="/Users/esmasert/Desktop/LungTrainingCodes/Cross_Validation_Data_November-2024/only_cross_validation_test_files/CV1/test/img/"
resultCSVFile = "/Users/esmasert/Desktop/PaperRelated/HeatMap/HeatmapInference/results/CV1/predictedPoints.csv"

    "segmentationModelPath = \"/data/Segmentation/chosen_ckpts/unet-6v_18th_epoch_CV1.pt\"\n",
heatMapModelPath = "/Users/esmasert/Desktop/PaperRelated/HeatMap/HeatMapCrossValTrainingResults/chosen_models2/weights-improvement-17_CV1.hdf5"


In [ ]:
sys.path.insert(0,'/data/src/')

445


# Getting Ground Truth Points

## Concatenate CSV files

In [ ]:
resultFolder="/data/HeatMap/HeatmapInference/results/CV1/"
testImageFolder="/data/Cross_Validation_Data_November-2024/only_cross_validation_test_files/CV1/test/img/"
resultCSVFile = "/data/HeatMap/HeatmapInference/results/CV1/predictedPoints.csv"

segmentationModelPath = "/data/Segmentation/chosen_ckpts/unet-6v_18th_epoch_CV1.pt"
heatMapModelPath = "/data/HeatMap/HeatMapCrossValTrainingResults/chosen_models2/weights-improvement-17_CV1.hdf5"

<ipython-input-4-0913e232470f>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfMain = dfMain.append(df, ignore_index=True)


In [5]:
len(dfMain)

2535

In [6]:
imgPoints=[]
count=0

for imageName in dfMain['image_name']:
    #print(imageName)
    res = any(imageName in imgname for imgname in test_files)
    #print(res)
    
    if res == True :
        count+=1

print("Number of test images: ",len(test_files))
print("Number of test images that are found in csv files: ",count)

Number of test images:  445
Number of test images that are found in csv files:  445


Some image files are not in csv files.

## Get The Points of Test Data

In [7]:
imgPoints=[]
for idx in range(0,len(dfMain)):
    res = any(dfMain['image_name'][idx] in imgname for imgname in test_files)
    if res == True :
        tmp=[]
        tmp.append(dfMain['image_name'][idx])
        tmp.append(int(dfMain['ref_point_fst_lung'][idx].split(', ')[0].split('(')[-1]))
        tmp.append(int(dfMain['ref_point_fst_lung'][idx].split(', ')[1].split(')')[0]))
        tmp.append(int(dfMain['ref_point_snd_lung'][idx].split(', ')[0].split('(')[-1]))
        tmp.append(int(dfMain['ref_point_snd_lung'][idx].split(', ')[1].split(')')[0]))
        imgPoints.append(tmp)

In [8]:
len(imgPoints)

445

In [9]:
imgPoints[0]

['146100695047215366253243105177291646230_yw3sa8', 153, 383, 375, 401]

# Predict with The Segmentation and HeatMap Model

We are going to predict the whole test data and saving the result points

In [10]:
unet = PretrainedUNet(1, 2, True, "bilinear")
index=0

/Users/esmasert/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/esmasert/opt/anaconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG11_Weights.IMAGENET1K_V1`. You can also use `weights=VGG11_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
unet.load_state_dict(torch.load(segmentationModelPath, map_location=torch.device("cpu")))
unet.to(device)
unet.eval();

In [ ]:
files = ['/data/CodesForCSVs/data/padchest_measurements.csv']

In [13]:
class DataGenerator(tf.keras.utils.Sequence):
    
    def __init__(self, imgs, batch_size=1, shuffle=True):
        self.imgs = imgs
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return len(self.imgs) // self.batch_size

    def __getitem__(self, index):
        #Get index of images to generate
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Generate data
        X = self.__data_generation(indexes)

        return X

    def on_epoch_end(self):
        #Shuffle the data after the generator has run through all samples
        self.indexes = np.arange(len(self.imgs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def gaussian(self, xL, yL, H, W, sigma=5):
        ##Function that creates the heatmaps##
        channel = [math.exp(-((c - xL) ** 2 + (r - yL) ** 2) / (2 * sigma ** 2)) for r in range(H) for c in range(W)]
        channel = np.array(channel, dtype=np.float32)
        channel = np.reshape(channel, newshape=(H, W))

        return channel

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'
        X_batch = [self.imgs for i in indexes]
        X_batch = np.array(X_batch)

        return X_batch

In [14]:
def sliding_window(image, stepSize, windowSize):
    windowAreaList=[]
    windowArea=0
    
    for x in range(0, image.shape[0]-windowSize, stepSize):
        for y in range(0, image.shape[1]-windowSize, stepSize):
            windowArea =0
            for num1 in range(0,windowSize):
                for num2 in range(0,windowSize):
                    #print(image[x+num1][y+num2])
                    windowArea+=image[x+num1][y+num2]
                    
            temp=[]
            temp.append(windowArea)
            temp.append(x)
            temp.append(y)
            
            windowAreaList.append(temp)
            
    return windowAreaList

In [15]:
def divideImageGetPoints(img):
    
    # Read the image
    height, width = img.shape

    # Cut the image in half
    width_cutoff = width // 2
    s1 = img[:, :width_cutoff]
    s2 = img[:, width_cutoff:]
    
    wwindowSize=10
    stepSize=5

    windowsLeft = sliding_window(s1, stepSize, wwindowSize)
    windowsRight = sliding_window(s2, stepSize, wwindowSize)
                
    #print("Len Windows Left:", len(windowsLeft))
    #print("Len Windows Right:", len(windowsLeft))
    #print("Len WindowSize:", wwindowSize)

    justWindowAreas=[]
    for i in windowsLeft:
        justWindowAreas.append(i[0])
    largest2nums = heapq.nlargest(1, enumerate(justWindowAreas), key=lambda x: x[1])
    #print(windowsLeft[largest2nums[0][0]])
    point1=windowsLeft[largest2nums[0][0]][2],windowsLeft[largest2nums[0][0]][1]
    cpoint1=int(point1[0]+(wwindowSize/2)),int(point1[1]+(wwindowSize/2))
    
    justWindowAreas=[]
    for i in windowsRight:
        justWindowAreas.append(i[0])
    largest2nums = heapq.nlargest(1, enumerate(justWindowAreas), key=lambda x: x[1])
    #print(windowsRight[largest2nums[0][0]])
    point2=windowsRight[largest2nums[0][0]][2],windowsRight[largest2nums[0][0]][1]
    cpoint2=int(point2[0]+(wwindowSize/2))+256,int(point2[1]+(wwindowSize/2))
    
    #print("Most intensive area centers: ",cpoint1,cpoint2)

    return cpoint1,cpoint2


In [16]:
def PredictModel(imagePath):
    
    origin = Image.open(imagePath).convert("P")
    origin = torchvision.transforms.functional.resize(origin, (512, 512))
    origin = torchvision.transforms.functional.to_tensor(origin) - 0.5
    
    with torch.no_grad():
        origin = torch.stack([origin])
        origin = origin.to(device)
        out = unet(origin)
        softmax = torch.nn.functional.log_softmax(out, dim=1)
        out = torch.argmax(softmax, dim=1)

        origin = origin[0].to("cpu")
        out = out[0].to("cpu")
        
    pil_origin = torchvision.transforms.functional.to_pil_image(origin + 0.5).convert("RGB")
    
    plt.imsave(resultFolder + "PredictedSegmentation.jpg" , out, cmap='gray')
    img = cv2.imread(resultFolder + "PredictedSegmentation.jpg")
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    stringg=""
    for row in gray:
        for itemIn in range(0,len(row)-1):
            stringg+=str(row[itemIn])+" "
        stringg+=str(row[itemIn+1])
        stringg+="\n"      
        
    img = stringg.split()
    img = ['0' if x == '' else x for x in img]

    image_list = np.array(img,dtype = 'float')
    X_train = image_list.reshape(512,512,1)
    
    #Testing to see if our DataGenerator is working
    X_batch = next(DataGenerator(X_train).__iter__())
    #print(X_batch.shape)

    HeatMapPredictions = heatMapModel.predict(X_batch, verbose=0)
    
    HeatMapresultImage= HeatMapPredictions[1][index].sum(axis=2)
    
    cpoint1, cpoint2 = divideImageGetPoints(HeatMapresultImage)
    
    return cpoint1,cpoint2, HeatMapresultImage, X_batch, pil_origin



In [17]:
resultImgPoints=[]
cnt=0

for imPath in tqdm(test_files):
    
    cpoint1, cpoint2, HeatMapresultImage, X_batch, pil_origin = PredictModel(imPath)

    justName = imPath.split("/")[-1].split(".")[0]
    #print(justName)
    
    tmp=[]
    tmp.append(justName)
    tmp.append(cpoint1[0])
    tmp.append(cpoint1[1])
    tmp.append(cpoint2[0])
    tmp.append(cpoint2[1])
    resultImgPoints.append(tmp)
    
    with open(resultCSVFile, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(tmp)
        
    # Turn interactive plotting off
    plt.ioff()
    
    fig = plt.figure()
    plt.imshow(X_batch.reshape(512,512))
    plt.plot(cpoint1[0], cpoint1[1], "o", markersize=10, color="red")  # og:shorthand for green circle
    plt.plot(cpoint2[0], cpoint2[1], "o", markersize=10, color="red")  # og:shorthand for green circle
    segName= resultFolder + justName + "_seg_hm.png"
    plt.savefig(segName)
    plt.close(fig)

    fig2 = plt.figure()
    plt.imshow(pil_origin)
    plt.plot(cpoint1[0], cpoint1[1], "o", markersize=10, color="red")  # og:shorthand for green circle
    plt.plot(cpoint2[0], cpoint2[1], "o", markersize=10, color="red")  # og:shorthand for green circle
    immName= resultFolder + justName + "_org_hm.png"
    fig2.savefig(immName)
    plt.close(fig2)
    
                 
    cnt+=1
    
    #if cnt==80:
    #    break


100%|██████████| 445/445 [59:55<00:00,  8.08s/it] 


In [18]:
TotalError = 0
icnt=0

for gtimg in imgPoints:
    for primg in resultImgPoints:
        
        if primg[0] == gtimg[0]:
            
            p1 = [gtimg[1], gtimg[2]]
            q1 = [primg[1], primg[2]]
            err1= math.dist(p1, q1)
            
            p2 = [gtimg[3], gtimg[4]]
            q2 = [primg[3], primg[4]]
            err2= math.dist(p2, q2)
            
            TotalError += err1
            TotalError += err2
            
            icnt+=1
            

In [19]:
icnt

445

In [20]:
ErrorRate = (TotalError / (icnt*512))

In [21]:
ErrorRate

0.04957721930650405

In [22]:
ErrorRate * 100

4.957721930650405